In [ ]:
from ultralytics import YOLO
from extract_final_save_from_client import extract_results_path

import pandas as pd

from FedYOLO.config import HOME, SPLITS_CONFIG, SERVER_CONFIG

DATASET_NAME = SPLITS_CONFIG['dataset_name']
NUM_ROUNDS = SERVER_CONFIG['rounds']

#! HUGE DIFFERENCES BETWEEN SYSTEMS REGARDING FILES PATHS AND LOGGING. NEED TO IN-DEPTH TEST THIS.

#####################
# CLIENT EVALUATION #
#####################

def get_client_metrics(client_number, dataset_name, home_path):
    # Extract paths
    print(home_path)
    client_model_weights_path = extract_results_path(f"{home_path}/logs/client_{client_number}_log_{dataset_name}.txt")
    weights = f"{home_path}/{client_model_weights_path}/weights/best.pt"
    
    # Load and validate local model
    client_model = YOLO(weights)
    client_metrics = client_model.val(data=f'{home_path}/datasets/{dataset_name}/partitions/client_{client_number}/data.yaml', verbose=False)
    
    # Create local model metrics table
    client_table = pd.DataFrame({
        'Class': list(client_metrics.names.values()),
        'mAP@0.5:0.95': client_metrics.box.maps.tolist()
    })
    
    # Extract global model weights
    client_global_model_weights_path = extract_results_path(f"{home_path}/logs/client_{client_number}_log_{dataset_name}.txt")
    global_weights = f"{home_path}/{client_global_model_weights_path}/weights/best.pt"
    
    # Load and validate global model
    client_global_model = YOLO(global_weights)
    client_global_metrics = client_global_model.val(data=f'{home_path}/datasets/{dataset_name}/data.yaml', verbose=False)
    
    # Create global model metrics table
    client_global_table = pd.DataFrame({
        'Class': list(client_global_metrics.names.values()),
        'mAP@0.5:0.95': client_global_metrics.box.maps.tolist()
    })
    
    # Combine local and global model metrics
    combined_table = pd.merge(client_table, client_global_table, on='Class', how='inner')
    combined_table.columns = ['Class', 'mAP@0.5:0.95_local', 'mAP@0.5:0.95_global']

    del client_model
    del client_global_model
    
    return combined_table

print('##################')
print('# CLIENT RESULTS #')
print('##################')
print()
client_0_metrics_table = get_client_metrics(0, DATASET_NAME, HOME)
client_1_metrics_table = get_client_metrics(1, DATASET_NAME, HOME)
combined_table = pd.merge(client_0_metrics_table, client_1_metrics_table, on='Class', how='inner')
combined_table.columns = ['Class', 'mAP@0.5:0.95_local_0', 'mAP@0.5:0.95_global_0', 'mAP@0.5:0.95_local_1', 'mAP@0.5:0.95_global_1']
print()
print()
print('##############################')
print('# FINAL CONSOLIDATED METRICS #')
print('##############################')
print(combined_table.to_string(index=False))
print()
print()

##################
# CLIENT RESULTS #
##################

/home/localssk23/UltraFlwr
Ultralytics 8.3.48 🚀 Python-3.12.2 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12800H)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


/home/localssk23/backup/soumya/env/fedlytics/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
val: Scanning /home/localssk23/UltraFlwr/datasets/baseline/partitions/client_0/valid/labels.cache... 6 images, 6 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.54it/s]


                   all         12         12    0.00177        0.3     0.0818     0.0412
Speed: 0.9ms preprocess, 44.1ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to runs/detect/val
Ultralytics 8.3.48 🚀 Python-3.12.2 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12800H)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /home/localssk23/UltraFlwr/datasets/baseline/valid/labels... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<00:00, 2473.11it/s]

val: New cache created: /home/localssk23/UltraFlwr/datasets/baseline/valid/labels.cache


...

                   all         25         48    0.00394      0.359     0.0596     0.0317
Speed: 0.7ms preprocess, 45.5ms inference, 0.0ms loss, 8.3ms postprocess per image
Results saved to runs/detect/val2
/home/localssk23/UltraFlwr
Ultralytics 8.3.48 🚀 Python-3.12.2 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12800H)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /home/localssk23/UltraFlwr/datasets/baseline/partitions/client_1/valid/labels.cache... 7 images, 6 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.34it/s]


                   all         13         13    0.00144      0.273     0.0556     0.0387
Speed: 0.9ms preprocess, 47.8ms inference, 0.0ms loss, 6.9ms postprocess per image
Results saved to runs/detect/val3
Ultralytics 8.3.48 🚀 Python-3.12.2 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12800H)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /home/localssk23/UltraFlwr/datasets/baseline/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


                   all         25         48    0.00348      0.359     0.0756     0.0457
Speed: 0.7ms preprocess, 45.2ms inference, 0.0ms loss, 10.0ms postprocess per image
Results saved to runs/detect/val4


##############################
# FINAL CONSOLIDATED METRICS #
##############################
  Class  mAP@0.5:0.95_local_0  mAP@0.5:0.95_global_0  mAP@0.5:0.95_local_1  mAP@0.5:0.95_global_1
cheetah              0.082355               0.063342              0.077394               0.091493
  human              0.000000               0.000000              0.000000               0.000000




In [8]:
import torch

#####################
# SERVER EVALUATION #
#####################
print('##################')
print('# SERVER RESULTS #')
print('##################')
print()

server_model = YOLO(f"{HOME}/yolo11n_{DATASET_NAME}.yaml")
server_model_weights_path = f"{HOME}/weights/model_round_{NUM_ROUNDS}_{DATASET_NAME}.pt"
server_model.model.load_state_dict(torch.load(server_model_weights_path)['model'].state_dict(), strict=False)

server_model_client0_metrics = server_model.val(data=f'{HOME}/datasets/{DATASET_NAME}/partitions/client_0/data.yaml', verbose=True)
server_model_client1_metrics = server_model.val(data=f'{HOME}/datasets/{DATASET_NAME}/partitions/client_1/data.yaml', verbose=True)
server_model_global_metrics = server_model.val(data=f'{HOME}/datasets/{DATASET_NAME}/data.yaml', verbose=True)

server_model_client0_table = pd.DataFrame({
    'Class': list(server_model_client0_metrics.names.values()),
    'mAP@0.5:0.95': server_model_client0_metrics.box.maps.tolist()
})

server_model_client1_table = pd.DataFrame({
    'Class': list(server_model_client1_metrics.names.values()),
    'mAP@0.5:0.95': server_model_client1_metrics.box.maps.tolist()
})

server_model_global_table = pd.DataFrame({
    'Class': list(server_model_global_metrics.names.values()),
    'mAP@0.5:0.95': server_model_global_metrics.box.maps.tolist()
})

# First, merge client0 and client1 tables
server_model_combined_table = pd.merge(server_model_client0_table, server_model_client1_table, on='Class', how='inner')

# Then, merge the result with the global table
server_model_combined_table = pd.merge(server_model_combined_table, server_model_global_table, on='Class', how='inner')

# Rename the columns
server_model_combined_table.columns = ['Class', 'mAP@0.5:0.95_client_0', 'mAP@0.5:0.95_client_1', 'mAP@0.5:0.95_global']

print(server_model_combined_table)

##################
# SERVER RESULTS #
##################

WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.48 🚀 Python-3.12.2 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12800H)
YOLO11n_baseline summary (fused): 238 layers, 2,582,542 parameters, 12,870 gradients, 6.3 GFLOPs


val: Scanning /home/localssk23/UltraFlwr/datasets/baseline/partitions/client_0/valid/labels.cache... 6 images, 6 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.56it/s]


                   all         12         12    0.00193       0.35     0.0832     0.0347
                     0          6         10    0.00386        0.7      0.166     0.0695
                     1          1          2          0          0          0          0
Speed: 0.5ms preprocess, 41.7ms inference, 0.0ms loss, 9.0ms postprocess per image
Results saved to runs/detect/val14
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.48 🚀 Python-3.12.2 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12800H)


val: Scanning /home/localssk23/UltraFlwr/datasets/baseline/partitions/client_1/valid/labels.cache... 7 images, 6 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.44it/s]


                   all         13         13    0.00176      0.318     0.0821      0.065
                     0          7         11    0.00351      0.636      0.164       0.13
                     1          1          2          0          0          0          0
Speed: 0.9ms preprocess, 43.8ms inference, 0.0ms loss, 6.9ms postprocess per image
Results saved to runs/detect/val15
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.48 🚀 Python-3.12.2 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12800H)


val: Scanning /home/localssk23/UltraFlwr/datasets/baseline/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.45it/s]


                   all         25         48    0.00368      0.359     0.0693     0.0429
                     0         25         39    0.00736      0.718      0.139     0.0858
                     1          3          9          0          0          0          0
Speed: 0.9ms preprocess, 44.4ms inference, 0.0ms loss, 7.3ms postprocess per image
Results saved to runs/detect/val16
  Class  mAP@0.5:0.95_client_0  mAP@0.5:0.95_client_1  mAP@0.5:0.95_global
0     0               0.069466               0.129926             0.085789
1     1               0.000000               0.000000             0.000000
